In [1]:
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

pd.options.mode.chained_assignment = None 

In [2]:
def recognizeSearcher(params, feats, featType, numQueryAnalysis):
    
    tnOva = 0
    fpOva = 0
    fnOva = 0
    tpOva = 0
    testAccOva = 0
    nSplits = 5
    x = 0
    z = 1
    accByNum = []
    outputAll = []
    outputQ = []

    kfold = KFold(n_splits=nSplits, random_state=20210530, shuffle=True)
    
    randomNumber = 20210530 #(open random number)
    
    X = SWC[feats]
    y = SWC['class']
   
    if params['scaler']:
        if params['classWeight']:
            tune_pipe = Pipeline([
            ('standardize', params['scaler']),
            ('classify', RandomForestClassifier(n_estimators=params['numEstimators'], bootstrap = params['bootStrap'],
                                                criterion= params['criterion'], class_weight = params['classWeight'], 
                                                random_state= randomNumber, n_jobs = -1))
            ])
        else:
                        tune_pipe = Pipeline([
            ('standardize', params['scaler']),
            ('classify', RandomForestClassifier(n_estimators=params['numEstimators'], bootstrap = params['bootStrap'],
                                                criterion= params['criterion'], random_state= randomNumber, n_jobs = -1))
            ])
    else:
        if params['classWeight']:
            tune_pipe = Pipeline([
            ('classify', RandomForestClassifier(n_estimators=params['numEstimators'], bootstrap = params['bootStrap'],
                                                criterion= params['criterion'], class_weight = params['classWeight'], 
                                                random_state= randomNumber, n_jobs = -1))
            ])
        else:
                        tune_pipe = Pipeline([
            ('classify', RandomForestClassifier(n_estimators=params['numEstimators'], bootstrap = params['bootStrap'],
                                                criterion= params['criterion'], random_state= randomNumber, n_jobs = -1))
            ])
                

    for train_index, test_index in kfold.split(X):

        rng = np.random.RandomState(randomNumber)
    
        trainSQS = SQS.sample(frac=0.20, random_state=rng)
#         train_mask = pd.Series(True, index=SQS.index)
#         train_mask[test.index] = False
#         train = SQS[train_mask].copy()
#         train.head()
        
        subTrain_x = trainSQS[feats]
        subTrain_y = trainSQS['class']
        
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        X_train = pd.concat([X_train,subTrain_x])
        y_train = pd.concat([y_train,subTrain_y])
        X_train = X_train.fillna(0)
        tune_pipe.fit(X_train, y_train)
        X_test['predict'] =  tune_pipe.predict(X_test)
        X_test['class'] = y_test
        testAccOva +=  accuracy_score(X_test['class'], X_test['predict'])
        tn, fp, fn, tp = confusion_matrix(X_test['class'], X_test['predict']).ravel()
        tnOva += tn
        fpOva += fp
        fnOva += fn
        tpOva += tp
        outputAll.append(X_test['predict'])
        if(numQueryAnalysis):

            accNum = []
            output = []
            ova = 0
            for x in range(5):
                testAcc = X_test[X_test['numQueries'] == (x+1)]
                acc = accuracy_score(testAcc['predict'], testAcc['class'])
                output.append(testAcc['predict'].tolist())
                SELtp = (testAcc[(testAcc['predict']  == 1) & (testAcc['class'] == 1)].index.tolist())
                tn, fp, fn, tp = confusion_matrix( testAcc['class'], testAcc['predict']).ravel()
                accNum.append([acc, tn, fp, fn, tp])
            testAcc = X_test[X_test['numQueries'] > (5)]
            acc = accuracy_score(testAcc['predict'], testAcc['class'])
            output.append(testAcc['predict'].tolist())
            SELtp = (testAcc[(testAcc['predict']  == 1) & (testAcc['class'] == 1)].index.tolist())
            tn, fp, fn, tp = confusion_matrix( testAcc['class'], testAcc['predict']).ravel()
            ova += ((tp +fn+fp+tn))
            accNum.append([acc, tn, fp, fn, tp])
            accByNum.append(accNum)
            outputQ.append(output)


        randomNumber +=1
        
    pickle.dump( outputAll, open( "Pickles/OutputAll" + str(featType) + "SWC.p", "wb" ) )
    results = [featType, round(testAccOva/5,3), tnOva/5, fpOva/5, round(((tnOva/5)/((tnOva/5)+ (fpOva/5))),3), fnOva/5, tpOva/5,round(((tpOva/5)/((tpOva/5)+ (fnOva/5))),3),]
    if(numQueryAnalysis):
        np.set_printoptions(suppress=True)
        acc1 = []
        acc2 = []
        acc3 = []
        acc4 = []
        acc5 = []
        acc6 = []

        for accBy in accByNum:
            x = 0
            for acc in accBy:
                if(x == 0):
                    acc1.append(acc)
                elif(x == 1):
                    acc2.append(acc)
                elif(x == 2):
                    acc3.append(acc)
                elif(x == 3):
                    acc4.append(acc)
                elif(x == 4):
                    acc5.append(acc)
                elif(x == 5):
                    acc6.append(acc)
                x+=1

                
        acc1 = np.array(acc1)
        acc1 = acc1.mean(axis = 0)
        acc2 = np.array(acc2)
        acc2 = acc2.mean(axis = 0)
        acc3 = np.array(acc3)
        acc3 = acc3.mean(axis = 0)
        acc4 = np.array(acc4)
        acc4 = acc4.mean(axis = 0)
        acc5 = np.array(acc5)
        acc5 = acc5.mean(axis = 0)
        acc6 = np.array(acc6)
        acc6 = acc6.mean(axis = 0)
        accAll = [acc1,acc2,acc3,acc4,acc5,acc6]
        accAll = pd.DataFrame(data = accAll, columns = ['Acc','TN','FP','FN','TP'])
        accAll['TNR'] = round((accAll['TN']/(accAll['TN']+accAll['FP'])),3)
        accAll['TPR'] = round((accAll['TP']/(accAll['TP']+accAll['FN'])),3)
        accAll = accAll.reset_index()
        accAll = accAll.rename(columns = {"index" :"numQ"})
        accAll['numQ'] = accAll['numQ']+1
        accAll['numQ'] = accAll['numQ'].astype(str)
        accAll['numQ'].replace({"6": "6+"}, inplace=True)
        accAll = accAll.set_index('numQ')
        pickle.dump(outputQ, open("Pickles/OutputByQueryCount.p", "wb"))
        pickle.dump(accAll, open("Pickles/AccByQueryCount.p", "wb"))
#         print(accAll)
#         print(results)
    return results

# Create Test and Train

In [3]:
SWC = pickle.load( open( "Pickles/SWCFeatNoTune.p", "rb" ) )
SQS =  pickle.load( open( "../FeatureExtraction/DataSets/SQSFeatures/SQSFeat.p", "rb" ) )
bestParameters = pickle.load( open( "Pickles/BestParam.p", "rb" ) )

In [4]:
features = list(SQS.columns)
features.remove('class')
features.remove('Filename')

In [5]:
features

['queryDistance',
 'timeQueries',
 'repeatQueries',
 'repeatClicks',
 'clickDistance',
 'meanClickPosition',
 'numClicks',
 'numClicksPerQuery',
 'numQueries',
 'timeClicks',
 'uniqueQueries',
 'allSameClicks',
 'uniqueClicks',
 'allSameQueries',
 'cc',
 'cd',
 'dt',
 'ex',
 'fw',
 'in',
 'jj',
 'jjr',
 'jjs',
 'ls',
 'md',
 'nn',
 'nnp',
 'nnps',
 'nns',
 'pdt',
 'pos',
 'prp',
 'rb',
 'rbr',
 'rbs',
 'rp',
 'sym',
 'to',
 'uh',
 'vb',
 'vbd',
 'vbg',
 'vbn',
 'vbp',
 'vbz',
 'wdt',
 'wp',
 'wrb',
 'nn nn',
 'jj nn',
 'nn nns',
 'to vb',
 'jj nns',
 'jj to',
 'nn in',
 'nns in',
 'in nn',
 'dt nn',
 'jj nn nn',
 'nn nn nn',
 'jj to vb',
 'nn nn nns',
 'to vb nn',
 ' Level0',
 ' Level1',
 ' Level2',
 ' Level3',
 ' Level4',
 ' Level5',
 ' Level6',
 ' Level7',
 ' MeanLevel',
 'totalSyl',
 'avgSyl',
 'simWords',
 'comWords',
 'greatestSyl',
 'leastSyl',
 'numChars',
 'numWords',
 'avgLenWord',
 'ld',
 'ls1',
 'ls2',
 'vs1',
 'vs2',
 'cvs1',
 'ndw',
 'ttr',
 'cttr',
 'rttr',
 'logttr',
 'l

In [6]:
P3Feats = ['ld',
 'ls1',
 'ls2',
 'vs1',
 'vs2',
 'cvs1',
 'ndw',
 'ttr',
 'cttr',
 'rttr',
 'logttr',
 'lv',
 'vv1',
 'svv1',
 'cvv1',
 'vv2',
 'nv',
 'adjv',
 'numSpellingErrors',
 'offByOne',
 'kidsError',
 'coreVocab',
 'nonCoreVocab',
 'vocabMin',
 'vocabMax',
 'queryComplexity',
 'addVocab',
 'SVEN',
 'top250All',
 'top250avgAll',
 'top250avgNotAll',
 'top250AllNA',
 'top250avgAllNA',
 'top250avgNotAllNA',
 'top50bi',
 'top50biAvg',
 'top50biNot',
 'top50biNA',
 'top50biavgNA',
 'top50biNotNA',
 'tfidfAll',
 'tfidf',
 'tfidfNA',
 ]

In [7]:
DC1Feats = ['ndw',
 'ttr',
 'cttr',
 'rttr',
 'logttr',
 'lv',
 'vv1',
 'svv1',
 'cvv1',
 'vv2',
 'nv',
 'adjv',
 'totalSyl',
 'avgSyl',
 'simWords',
 'comWords',
 'greatestSyl',
 'leastSyl',
 'numChars',
 'numWords',
 'avgLenWord',
 'vocabMin',
 'vocabMax',
 'queryComplexity',
 'stopCount',
 'com',
 'net',
 'org',
 'edu',
 'gov',
 'http',
 'AND',
 'OR',
 'quotes',
 'inter',
 'numSpellingErrors',
 'offByOne',
 'kidsError',
 'punct',
 'casing',
 'cc',
 'cd',
 'dt',
 'ex',
 'fw',
 'in',
 'jj',
 'jjr',
 'jjs',
 'ls',
 'md',
 'nn',
 'nnp',
 'nnps',
 'nns',
 'pdt',
 'pos',
 'prp',
 'rb',
 'rbr',
 'rbs',
 'rp',
 'sym',
 'to',
 'uh',
 'vb',
 'vbd',
 'vbg',
 'vbn',
 'vbp',
 'vbz',
 'wdt',
 'wp',
 'wrb',
 'nn nn',
 'jj nn',
 'nn nns',
 'to vb',
 'jj nns',
 'jj to',
 'nn in',
 'nns in',
 'in nn',
 'dt nn',
 'jj nn nn',
 'nn nn nn',
 'jj to vb',
 'nn nn nns',
 'to vb nn',]

In [8]:
TextFeat = ['cc',
 'cd',
 'dt',
 'ex',
 'fw',
 'in',
 'jj',
 'jjr',
 'jjs',
 'ls',
 'md',
 'nn',
 'nnp',
 'nnps',
 'nns',
 'pdt',
 'pos',
 'prp',
 'rb',
 'rbr',
 'rbs',
 'rp',
 'sym',
 'to',
 'uh',
 'vb',
 'vbd',
 'vbg',
 'vbn',
 'vbp',
 'vbz',
 'wdt',
 'wp',
 'wrb',
 'nn nn',
 'jj nn',
 'nn nns',
 'to vb',
 'jj nns',
 'jj to',
 'nn in',
 'nns in',
 'in nn',
 'dt nn',
 'jj nn nn',
 'nn nn nn',
 'jj to vb',
 'nn nn nns',
 'to vb nn',
 ' Level0',
 ' Level1',
 ' Level2',
 ' Level3',
 ' Level4',
 ' Level5',
 ' Level6',
 ' Level7',
 ' MeanLevel',
 'totalSyl',
 'avgSyl',
 'simWords',
 'comWords',
 'greatestSyl',
 'leastSyl',
 'numChars',
 'numWords',
 'avgLenWord',
 'ld',
 'ls1',
 'ls2',
 'vs1',
 'vs2',
 'cvs1',
 'ndw',
 'ttr',
 'cttr',
 'rttr',
 'logttr',
 'lv',
 'vv1',
 'svv1',
 'cvv1',
 'vv2',
 'nv',
 'adjv',
 'numSpellingErrors',
 'offByOne',
 'kidsError',
 'punct',
 'casing',
 'coreVocab',
 'nonCoreVocab',
 'vocabMin',
 'queryComplexity',
 'addVocab',
 'vocabMax',
 'SVEN',
 'top250All',
 'top250avgAll',
 'top250avgNotAll',
 'top250AllNA',
 'top250avgAllNA',
 'top250avgNotAllNA',
 'top50bi',
 'top50biAvg',
 'top50biNot',
 'top50biNA',
 'top50biavgNA',
 'top50biNotNA',
 'tfidfAll',
 'tfidf',
 'tfidfNA',
 'stopCount',
 'com',
 'net',
 'org',
 'edu',
 'gov',
 'http',
 'AND',
 'OR',
 'quotes',
 'inter']

In [9]:
SessionFeats = ['repeatClicks',
 'clickDistance',
 'meanClickPosition',
 'numClicks',
 'numClicksPerQuery',
 'numQueries',
 'timeClicks',
 'uniqueQueries',
 'allSameClicks',
 'uniqueClicks',
 'allSameQueries',
 'queryDistance',
 'timeQueries',
 'repeatQueries']

In [10]:
SWC[SWC['numQueries']==0]

,class,cc,cd,dt,ex,fw,in,jj,jjr,jjs,...,numClicksPerQuery,numQueries,timeClicks,uniqueQueries,allSameClicks,uniqueClicks,allSameQueries,queryDistance,timeQueries,repeatQueries
sID,,,,,,,,,,,,,,,,,,,,,


In [11]:
RYSe = recognizeSearcher(bestParameters, features, 'RYSe', True)
P3Results = recognizeSearcher(bestParameters, P3Feats, 'P3', False)
DC1Results = recognizeSearcher(bestParameters, DC1Feats, 'DC1', False)
textResults = recognizeSearcher(bestParameters, TextFeat, 'TextBased', False)
sessionResults = recognizeSearcher(bestParameters, SessionFeats, 'SessionBased', False)

In [12]:
print(RYSe)
print(P3Results)
print(DC1Results)
print(textResults)
print(sessionResults)

['RYSe', 0.952, 5219.6, 56.2, 0.989, 258.0, 1065.8, 0.805]
['P3', 0.94, 5182.8, 93.0, 0.982, 303.6, 1020.2, 0.771]
['DC1', 0.888, 5203.6, 72.2, 0.986, 668.4, 655.4, 0.495]
['TextBased', 0.937, 5193.4, 82.4, 0.984, 334.2, 989.6, 0.748]
['SessionBased', 0.857, 4893.4, 382.4, 0.928, 562.6, 761.2, 0.575]


In [13]:
allResults = pd.DataFrame(data = [RYSe,P3Results,DC1Results,textResults,sessionResults  ], columns = ["Type","Acc", "TN", "FP", "TNR", "FN", "TP", "TPR"])

In [14]:
pickle.dump( allResults, open( "Pickles/AblationSWC.p", "wb" ) )